# 🔧 Manual Tool Integration vs 🚀 MCP Server – Demonstration Notebook
This notebook shows **how much effort is needed to build & integrate tools manually**, and how **MCP eliminates all that pain**.

We will:
1. Build a **manual tool** (Fetch Weather API)
2. Show the **integration pain** when using it with different LLM frameworks
3. Show the **same tool wrapped as an MCP server**, which becomes instantly reusable


## 1️⃣ Manual Tool – Build a Weather Fetcher
This is how developers normally write tools.

In [ ]:
import requests

def get_weather(city: str):
    url = f"https://wttr.in/{city}?format=j1"
    data = requests.get(url).json()
    return {
        "city": city,
        "temp": data['current_condition'][0]['temp_C'],
        "humidity": data['current_condition'][0]['humidity'],
    }

get_weather("Hyderabad")

## 2️⃣ Pain Point – Using This Tool With Different LLM Frameworks 
For **every LLM**, you must write a CUSTOM WRAPPER for this tool:
- OpenAI tool schema wrapper
- Claude tool schema wrapper
- LangChain tool wrapper
- LangGraph tool node
- CrewAI agent tool plugin

❌ Every framework wants tools in different formats.
❌ You maintain 5 wrappers for 1 tool.
❌ Scaling becomes impossible.


### Example – OpenAI Tool Wrapper (custom code)


In [ ]:
openai_schema = {
    "name": "get_weather",
    "description": "Fetch weather for a city",
    "parameters": {
        "type": "object",
        "properties": {"city": {"type": "string"}},
        "required": ["city"],
    }
}

openai_schema

### Example – LangChain Wrapper (more boilerplate)


In [ ]:
class LCWeatherTool:
    name = "lc_weather"
    description = "LC tool wrapper"
    
    def _run(self, city: str):
        return get_weather(city)

LCWeatherTool()

📌 **This duplication is the REAL problem.**

You build ONE tool… but you must maintain FIVE wrappers.
When you update the tool → you must update ALL wrappers.
This is not scalable.


# 🚀 3️⃣ Now the Magic – MCP Server Version of the SAME Tool
An MCP server standardizes:
- tool schema
- tool discovery
- tool invocation
- transport (STDIO/SSE)

**One server → all LLMs can use it. No wrappers. No rewrites.**

In [ ]:
# Pseudo-code MCP server definition
mcp_tool_schema = {
    "name": "get_weather",
    "description": "Get weather via MCP server",
    "input_schema": {"city": "string"},
    "output_schema": {
        "city": "string",
        "temp": "string",
        "humidity": "string",
    }
}
mcp_tool_schema

## 💥 And now ANY LLM can instantly use this tool:
- Claude Desktop
- OpenAI Agents
- Cursor
- VSCode MCP Clients
- Browser MCP playgrounds

All they do is:
```
client.get_tools()
client.call_tool("get_weather", { city: "Hyderabad" })
```
🎉 **ZERO wrapper code needed.**